In [29]:
import os
import yaml
import pandas as pd

from pathlib import Path
# from sklearn.metrics import ConfusionMatrixDisplay

from ba_dev.eval_helpers import *

set_custom_plot_style()

In [2]:
path_to_models = Path('/cfs/earth/scratch/kraftjul/BA/output/complete')

model_paths = [p for p in path_to_models.iterdir() if p.is_dir()]

In [3]:
model_paths[0]

PosixPath('/cfs/earth/scratch/kraftjul/BA/output/complete/resNet50_v1_no_pretrained_cross_val')

In [4]:
run = LoadRun(
        log_path=model_paths[0]
        )

In [ ]:
bal_accuracy_testing = []

for fold in range(5):
    value = run.info['output']['folds'][fold]['test_metrics']['test_bal_acc']
    bal_accuracy_testing.append(value)

In [5]:
metric_img = run.calculate_metrics(
    metric='balanced_accuracy_score',
    set_selection='test',
    scope='img'
    )

In [6]:

metric_seq = run.calculate_metrics(
    metric='balanced_accuracy_score',
    set_selection='test',
    scope='seq'
    )

In [7]:
metric_img

[0.9841004892822163,
 0.9885165416837002,
 0.9846517401351076,
 0.9796977764908908,
 0.9877960492571849]

In [8]:
metric_seq

[0.987187361318395,
 0.9876940629307991,
 0.9904345957378272,
 0.9849416190715843,
 0.9936135676143917]

In [25]:
all_models = []
for model_path in model_paths:
    yaml_path = model_path / 'experiment_info.yaml'
    if not yaml_path.exists():
        raise FileNotFoundError(f"Experiment info file not found at {yaml_path}")
    with open(yaml_path, 'r') as f:
        info = yaml.safe_load(f)
    all_models.append(info)

In [ ]:
path_to_models = Path('/cfs/earth/scratch/kraftjul/BA/output/complete')
model_paths = list(path_to_models.glob('*/'))

all_metrics = []
for model_path in model_paths:
    model = LoadRun(
        log_path=model_path
        )
    
    n_folds = model.info['cross_val']['n_folds']
    model_name = model.info['model']['backbone_name']
    pretrained = model.info['model']['backbone_pretrained']
    trainable_params = model.info['output']['model_parameters']['trainable']
    
    metric_img = model.calculate_metrics(
        metric='balanced_accuracy_score',
        set_selection='test',
        scope='img'
        )
    
    metric_seq = model.calculate_metrics(
        metric='balanced_accuracy_score',
        set_selection='test',
        scope='seq'
        )
    
    for fold in range(n_folds):
        item = {
                'model_name':   model_name,
                'pretrained':   pretrained,
                'trainable_params': trainable_params,
                'fold': fold,
                'bal_acc_image':   metric_img[fold],
                'bal_acc_seq':     metric_seq[fold],
                }
        
        all_metrics.append(item)
    
df = pd.DataFrame(all_metrics)
